In [1]:
import pandas as pd
from gensim.models.fasttext import FastText

Reading in training data


In [86]:
df1 = pd.read_csv('finnum/train.csv')
x = df1.index
df2 = pd.read_csv('finnum/dev.csv')
df = pd.concat([df1,df2], ignore_index = True)
df.head(10)

,id,idx,tweet,category,subcategory,target_num
0,98221616,4976,$ARNA APD334 for Amyotrophic Lateral Sclerosis...,Product Number,Product Number,334
1,82321187,9839,"$OCLR Noob investor that i am, put a 7.38 stop...",Monetary,stop loss,7.38
2,103328840,1455,$ES_F $SPY Bias-2 bearish and the DLT-1 DRR ar...,Product Number,Product Number,1
3,104840294,1111,$TMUS its acquisition of Layer3 TV The purchas...,Product Number,Product Number,5
4,69935467,2373,$TWTR ^Buy $WSTL 68c up 14% 4 time avg vol. ...,Percentage,relative,14
5,69935467,2373,$TWTR ^Buy $WSTL 68c up 14% 4 time avg vol. ...,Quantity,Quantity,4
6,69935467,2373,$TWTR ^Buy $WSTL 68c up 14% 4 time avg vol. ...,Monetary,forecast,5
7,94249158,1372,$SEED L2 Capital deal is real savvy. It takes ...,Temporal,date,33
8,100979260,505,$BTE $BTE.CA $MEG.CA $CPG $CPG.CA $CJ.CA - 4th...,Temporal,date,4
9,100775772,1210,$WRN My fav $WRN pattern on my watchlist for 1...,Temporal,date,11


Make new column for encoding categories as numbers

In [87]:
df['cat_num'] = df['category'].astype('category').cat.codes

Loading in fastText model

In [89]:
model = FastText.load('fastText1')

Preprocessing tweets by lowercasing

In [90]:
df['lower'] = [x.lower() for x in df.tweet]

In [91]:
df['lower'].head()

0    $arna apd334 for amyotrophic lateral sclerosis...
1    $oclr noob investor that i am, put a 7.38 stop...
2    $es_f $spy bias-2 bearish and the dlt-1 drr ar...
3    $tmus its acquisition of layer3 tv the purchas...
4    $twtr ^buy  $wstl 68c up 14%  4 time avg vol. ...
Name: lower, dtype: object

Replacing target with <num\>

In [92]:
import re
def flagNum(x):
    text = x.lower
    outNum = str(x.target_num)
    text_out = re.sub(r'(?<=\D)'+outNum+'(?=\D)', ' <num> ', text)
    #text_out = text.replace('\D('+outNum+')\D', ' <num> ')
    return text_out

Applying to training, making this into a new column

In [93]:
df['mod'] = df.apply(lambda x: flagNum(x), axis = 1)

In [94]:
df['mod'].head()

0    $arna apd <num>  for amyotrophic lateral scler...
1    $oclr noob investor that i am, put a  <num>  s...
2    $es_f $spy bias-2 bearish and the dlt- <num>  ...
3    $tmus its acquisition of layer3 tv the purchas...
4    $twtr ^buy  $wstl 68c up  <num> %  4 time avg ...
Name: mod, dtype: object

In [95]:
df.lower.head()

0    $arna apd334 for amyotrophic lateral sclerosis...
1    $oclr noob investor that i am, put a 7.38 stop...
2    $es_f $spy bias-2 bearish and the dlt-1 drr ar...
3    $tmus its acquisition of layer3 tv the purchas...
4    $twtr ^buy  $wstl 68c up 14%  4 time avg vol. ...
Name: lower, dtype: object

Filtering out numbers and words less than 3 characters long

In [96]:
import string
def textPuncandNum(text):
    table = str.maketrans('', '', string.punctuation)
    text = re.sub(r'[0-9]+', '', text)
    text = text.split()
    text = [word for word in text if len(word.translate(table))>2]
    return ' '.join(text)
stripped = [textPuncandNum(text) for text in df['mod']]

Creating list of unique words from this processed text, excluding <num\>

In [97]:
low = list(stripped)
low = ' '.join(low)
low = list(set(low.split()))
low.remove('<num>')
len(low)

14064

In [98]:
low

['correction:',
 'robotic',
 'lmao!',
 'expansion',
 'uptake',
 'held.',
 'hit.',
 'why.',
 '$bur',
 'https://finance.yahoo.com/news/credit-suisse-ag-announces-reverse-.html',
 'cryptocurrency',
 'c-suite',
 'slv',
 'you?',
 'http://stks.co/tbnu',
 '$nxtd',
 '$snss',
 'erinn...looks',
 'japan.',
 'wasn&#;t',
 'metrics',
 'days...',
 'delux,',
 '$tfm',
 '$gpor',
 '$amrs-',
 'force.',
 'gtav',
 '@tfnn',
 'cpu',
 'lunch.',
 'doubted',
 '$cmm.v',
 '$omgbtc',
 '$sgoc',
 '$exk',
 'mainstream',
 'ipdn',
 '$viab',
 'capita',
 'instant',
 'cheaper',
 'needham.',
 'traffic',
 'casino.',
 'would',
 'goiing',
 'gdpr',
 '$fmsa',
 'establish',
 'agen',
 'agrawals',
 'price.',
 'quickly?',
 'gauge',
 '$jwn.',
 'pricey',
 'limits',
 'confident.',
 'launch.',
 'amzn.',
 'double+',
 'https://www.thecontraaccountant.com/single-post////have-you-forgotten-about-the-',
 'fool.',
 'reforms.',
 '$million',
 'sears',
 '@njdevilsinseven',
 'cycle',
 'reach',
 'excellent',
 'tomorrow....capisce?',
 '$aprn',
 '&q

Using model to get embeddingss for these words

In [99]:
embed = model.wv[low]

In [100]:
embed.shape

(14064, 100)

Indexing these unqique words in a dictionary 

In [101]:
label_dict = {k: v for v, k in enumerate(low)}
label_dict['<num>'] = len(label_dict) 

Adding <num\> back in 

In [102]:
label_dict['<num>']

14064

Adding filler word to keep all tweets the same length. Then replacing all words with their dictionary equivalent. This is for tenssorflows matrix creation

In [103]:
bufferIndex = len(label_dict) 

In [104]:
import numpy as np
modifiedText = [[label_dict[word] for word in text.split()]for text in stripped]
maxLen = max(map(len, modifiedText))
for item in modifiedText:                # for each item in the list
    while len(item) < maxLen:            # while the item length is smaller than maxLen
        item.append(bufferIndex) 
numpyInp = np.asarray(modifiedText)

In [105]:
numpyInp

array([[14052,  9307, 14064, ..., 14065, 14065, 14065],
       [12214,   792, 10889, ..., 14065, 14065, 14065],
       [ 4081,  2743,   212, ..., 14065, 14065, 14065],
       ...,
       [ 7248, 12383, 13982, ..., 14065, 14065, 14065],
       [ 9059, 13030,  4020, ..., 14065, 14065, 14065],
       [ 7050,  3231, 12104, ..., 14065, 14065, 14065]])

Adding in unique embeddings for <num\> and filler

In [106]:
embed = np.vstack((embed, np.zeros(100)+20, np.zeros(100)+25))

In [107]:
embed.shape

(14066, 100)

In [112]:
train = numpyInp[x,:]
val = numpyInp[len(x):,:]

In [115]:
val.shape[0] + train.shape[0]

7450

In [48]:
embed2.shape

(2952, 100)

## Implementing Convolutional Layer

In [142]:
import tensorflow as tf
batch_size = 16

In [214]:
tf.reset_default_graph()

Creating the embedding matrix for any input


In [215]:
trainInp = numpyInp[x,:]
valInp = numpyInp[len(x):,:]

labels = pd.get_dummies(df['cat_num'][x])
labels2 = pd.get_dummies(df['cat_num'][len(x):])

Batching and creating iterators

In [216]:
train = (trainInp, labels)
val = (valInp, labels2)

# create training Dataset and batch it
train_data = tf.data.Dataset.from_tensor_slices(train)
train_data = train_data.shuffle(10000) # if you want to shuffle your data
train_data = train_data.batch(batch_size)

# create validation Dataset and batch it
val_data = tf.data.Dataset.from_tensor_slices(val)
val_data = val_data.shuffle(10000) # if you want to shuffle your data
val_data = val_data.batch(batch_size)

# create one iterator and initialize it with different datasets
iterator = tf.data.Iterator.from_structure(train_data.output_types, 
                                           train_data.output_shapes)
txt, label = iterator.get_next()

train_init = iterator.make_initializer(train_data)
val_init = iterator.make_initializer(val_data)

In [217]:
embedding = tf.nn.embedding_lookup(embed, txt, partition_strategy='mod', name=None)
embedded_chars_expanded = tf.expand_dims(embedding, -1)

What does enumerate do?


In [218]:
filter_sizes = [0,3,4]
for i in enumerate(filter_sizes):
    print(i)

(0, 0)
(1, 3)
(2, 4)


Failed attempt using predefined filter
Update: Works now, not using tho

In [219]:
'''
WINDOW_SIZE = 100
STRIDE = int(WINDOW_SIZE/2)
#embedding2 = tf.expand_dims(embedding, axis = 1)
conv = tf.layers.conv2d(embedded_chars_expanded, 2, [2,WINDOW_SIZE], 
               strides=1, padding='SAME') 
conv = tf.nn.relu(conv)   
words = flatten(conv)
'''

"\nWINDOW_SIZE = 100\nSTRIDE = int(WINDOW_SIZE/2)\n#embedding2 = tf.expand_dims(embedding, axis = 1)\nconv = tf.layers.conv2d(embedded_chars_expanded, 2, [2,WINDOW_SIZE], \n               strides=1, padding='SAME') \nconv = tf.nn.relu(conv)   \nwords = flatten(conv)\n"

In [220]:
embedding

<tf.Tensor 'embedding_lookup/Identity:0' shape=(?, 26, 100) dtype=float64>

In [221]:
embedded_chars_expanded

<tf.Tensor 'ExpandDims:0' shape=(?, 26, 100, 1) dtype=float64>

Implementing looped convolution

In [222]:
pooled_outputs = []
filter_sizes = [2,3,4,5]
embedding_size = 100
num_filters = 64
max_length = 26
for filter_size in filter_sizes:
    filter_shape = [filter_size, embedding_size, 1, num_filters]
    W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name='W')
    b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name = 'b')
    conv = tf.nn.conv2d(
        embedded_chars_expanded,
        tf.cast(W,tf.float64),
        strides=[1, 1, 1, 1],
        padding='VALID',
        name='conv')
    relu = tf.nn.relu(tf.nn.bias_add(conv, tf.cast(b,tf.float64)), name="relu")
    pooled = tf.nn.max_pool(
        relu,
        ksize=[1, max_length - filter_size + 1, 1, 1],
        strides=[1, 1, 1, 1],
        padding='VALID',
        name="pool")
    pooled_outputs.append(pooled)

In [223]:
pooled_outputs

[<tf.Tensor 'pool:0' shape=(?, 1, 1, 64) dtype=float64>,
 <tf.Tensor 'pool_1:0' shape=(?, 1, 1, 64) dtype=float64>,
 <tf.Tensor 'pool_2:0' shape=(?, 1, 1, 64) dtype=float64>,
 <tf.Tensor 'pool_3:0' shape=(?, 1, 1, 64) dtype=float64>]

Combining separate convolutional layers into 1 feed forward input

In [224]:
num_filters_total = num_filters * len(filter_sizes)
combined = tf.concat(pooled_outputs, 3)
combined_flat = tf.reshape(combined, [-1, num_filters_total])

In [225]:
combined_flat

<tf.Tensor 'Reshape:0' shape=(?, 256) dtype=float64>

Adding dense layers

In [226]:
conn = tf.layers.dense(combined_flat, 100, activation = 'relu')
conn2 = tf.layers.dense(conn, len(set(df.cat_num)))

In [227]:
conn

<tf.Tensor 'dense/Relu:0' shape=(?, 100) dtype=float64>

Implementing cross entropy, loss, and optimization

In [228]:
n_epochs = 50
entropy = tf.nn.softmax_cross_entropy_with_logits(labels = label, logits = conn2)
loss = tf.reduce_mean(entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = 0.0001).minimize(loss)

Prediction setup

In [229]:
preds = tf.nn.softmax(conn2)
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(label, 1))
accuracy = tf.reduce_mean(tf.cast(correct_preds, tf.float32))

Running Neural Net

In [231]:
with tf.Session() as sess:
    #start_time = time.time()
    sess.run(tf.global_variables_initializer())
    # train the model n_epochs times

    for i in range(n_epochs): 
        
        sess.run(train_init)# drawing samples from train_data
        total_loss = 0
        total_right = 0
        n_batches = 0
        totalright = 0
        totalvalright = 0
        nvalbatches = 0
        totalvalloss = 0
        try:
            while True:
                #summary,acc,_, l = sess.run([summary_op,accuracy,optimizer, loss]) #use with scalar summary
                acc,_, l = sess.run([accuracy, optimizer, loss])                
                total_loss += l
                total_right += acc
                n_batches += 1
        except tf.errors.OutOfRangeError:
            pass
        
        sess.run(val_init)
        try:
            while True:
                accval,valloss = sess.run([accuracy, loss])
                totalvalright += accval
                nvalbatches += 1
                totalvalloss += valloss
        except tf.errors.OutOfRangeError:
            pass

        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
        print('Accuracy {0}: {1}'.format(i, total_right/n_batches))    
        print('Average val loss epoch {0}: {1}'.format(i, totalvalloss/nvalbatches))
        print('Val_Accuracy {0}: {1}'.format(i, totalvalright/nvalbatches)) 
    prediction = sess.run(preds, feed_dict={txt: valInp})
    prediction = np.asarray(prediction)
   


Average loss epoch 0: 4.007726843413243
Accuracy 0: 0.3325892857142857
Average val loss epoch 0: 2.783648899751975
Val_Accuracy 0: 0.28856382978723405
Average loss epoch 1: 2.384174376319691
Accuracy 1: 0.41101190476190474
Average val loss epoch 1: 3.0805691838945912
Val_Accuracy 1: 0.4002659574468085
Average loss epoch 2: 1.888114726271596
Accuracy 2: 0.46383928571428573
Average val loss epoch 2: 1.8698339953724206
Val_Accuracy 2: 0.4773936170212766
Average loss epoch 3: 1.7440483381382665
Accuracy 3: 0.49404761904761907
Average val loss epoch 3: 2.265401900711219
Val_Accuracy 3: 0.37632978723404253
Average loss epoch 4: 1.5518345099943864
Accuracy 4: 0.5300595238095238
Average val loss epoch 4: 2.243899756778446
Val_Accuracy 4: 0.4601063829787234
Average loss epoch 5: 1.4226784007160802
Accuracy 5: 0.546875
Average val loss epoch 5: 2.191701245306981
Val_Accuracy 5: 0.46941489361702127
Average loss epoch 6: 1.3149245895284956
Accuracy 6: 0.578125
Average val loss epoch 6: 1.749435159

Calculating accuracy for train

In [240]:
np.mean(np.equal(np.argmax(prediction, 1), labels2.idxmax(axis = 1)))

0.5403225806451613

Fooling around with numpy

In [ ]:
f = np.array([[[1,2,3], [1,2,3]], [[1,2,3], [1,2,3]], [[1,2,3], [1,2,3]]])
#f = np.array([[1,2,3], [1,2,3]])
#print(f)
g = np.expand_dims(f, axis = 1)
g = np.reshape(f, [-1,2])
print(g)
print(g.shape)
print(np.squeeze(g))